<a href="https://colab.research.google.com/github/NicolasPinell/Backtesting/blob/master/frames9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install statsbomb

In [2]:
import statsbomb as sb
print(sb)
comps = sb.Competitions()
print(len(comps))  # 3
json_data = comps.data  # underlying json data

comps_df = comps.get_dataframe()

print(comps_df.columns)

print(comps_df[['season_id','season_name','competition_name','match_available']])

<module 'statsbomb' from '/usr/local/lib/python3.6/dist-packages/statsbomb/__init__.py'>
20
Index(['competition_id', 'season_id', 'country_name', 'competition_name',
       'competition_gender', 'season_name', 'match_updated',
       'match_available'],
      dtype='object')
    season_id season_name         competition_name             match_available
0          42   2019/2020  FA Women's Super League  2020-05-10T17:05:18.926659
1           4   2018/2019  FA Women's Super League  2020-04-23T19:04:49.880742
2           3        2018           FIFA World Cup  2019-12-16T23:09:16.168756
3           4   2018/2019                  La Liga  2020-04-21T21:40:19.168478
4           1   2017/2018                  La Liga  2020-05-03T17:05:26.772937
5           2   2016/2017                  La Liga  2019-12-16T23:09:16.168756
6          27   2015/2016                  La Liga  2020-04-13T23:02:59.803428
7          26   2014/2015                  La Liga  2019-12-16T23:09:16.168756
8          25

In [3]:
matches = sb.Matches(event_id='11', season_id='37')
df = matches.get_dataframe()
print(df.columns)  # 7
print(df['metadata'][1])

Index(['match_id', 'match_date', 'kick_off', 'competition', 'season',
       'home_team', 'away_team', 'home_score', 'away_score', 'match_status',
       'last_updated', 'metadata', 'match_week', 'competition_stage',
       'stadium', 'referee'],
      dtype='object')
{'data_version': '1.1.0', 'shot_fidelity_version': '2', 'xy_fidelity_version': '2'}


In [4]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
events = sb.Events(event_id='8658')
df = json_normalize(vars(events)['data'], sep = "_")
df1 = df
#pd.set_option('display.max_columns', None)
#print(df.head())
#print(type(df[df['shot_end_location'].notnull()]['shot_end_location']))

df['pass_outcome_name'] = df['pass_outcome_name'].replace(np.nan, "Complete")

print(df['pass_outcome_id'].unique())
print(df['pass_outcome_name'].unique())

df[['location_x', 'location_y']] = df['location'].apply(pd.Series)
df[['pass_end_location_x', 'pass_end_location_y']] = df['pass_end_location'].apply(pd.Series)
#print(df.head())

df['shot_end_location_x'], df['shot_end_location_y'], df['shot_end_location_z'] = np.nan, np.nan, np.nan
end_locations = np.vstack(df.loc[df.type_name == 'Shot'].shot_end_location.apply(lambda x: x if len(x) == 3
                                       else x + [float("nan")]).values)
df.loc[df.type_name == 'Shot', 'shot_end_location_x'] = end_locations[:, 0]
df.loc[df.type_name == 'Shot', 'shot_end_location_y'] = end_locations[:, 1]
df.loc[df.type_name == 'Shot', 'shot_end_location_z'] = end_locations[:, 2]
df.drop(['location', 'pass_end_location', 'shot_end_location'], axis = 1)
#df = df.fillna(0)
df['location_x'] = df['location_x'].replace(np.nan, float("nan"))
df['location_y'] = df['location_y'].replace(np.nan, float("nan"))

df.loc[df['shot_end_location_x'].notnull(), 'shot_location_x'] = df['location_x']
df.loc[df['shot_end_location_y'].notnull(), 'shot_location_y'] = df['location_y']

df.loc[df['pass_end_location_x'].notnull(), 'pass_location_x'] = df['location_x']
df.loc[df['pass_end_location_y'].notnull(), 'pass_location_y'] = df['location_y']


#print(lambda arrival_time: counter+1 if outcome == None else counter)(df)
#print('counter',counter)
#c = [n for n in filter(lambda x: counter+1 if df['outcome'] == None else counter, df['outcome'])]

#counter = df['pass_outcome_name'].apply(lambda x: counter+1 if x == 'Complete' else counter)

#print(counter.to_list())

#df.assign(lambda x: x['play'] * 9 / 5 + 32)

#df['play'] = 0
#j = 0
#for i, outcome in enumerate(df['pass_outcome_name']):
    
#    if df['pass_end_location_x'][i] != 'nan':
#        print(df['pass_end_location_x'][i])
#        if outcome == None:
#            df['play'][i] = j
#        else:
#            j += 1
#            df['play'][i] = j


#l = df[df['pass_shot_assist'].notnull()]['play'].to_list()
#for i in l: 
    #df.loc[df.play == i, 'shot_assist'] = True
    #df2[df2['play']==i]['shot_assist']=True
#print(df['shot_assist'])

#df['shot_location_x'] = np.where(df['shot_end_location_x'].notnull(), df['location_x'], 'NaN')
#df['shot_location_y'] = np.where(df['shot_end_location_y'].notnull(), df['location_y'], 'NaN')
df = df.fillna(float("nan"))

#j = 0
#df['play'] = np.nan
#print('play',df['play'])
#for i, outcome in enumerate(df['pass_outcome_name']):
    
    #if df['pass_end_location_x'][i] != 'nan':
        #print(df['pass_end_location_x'][i])
#    if outcome == 'Complete':
#        df['play'][i] = j
#    else:
#        j += 1
#        df['play'][i] = j
pd.set_option('display.max_columns', None)
print(df.columns.values)

l = df[df['pass_shot_assist']==True][['possession','pass_assisted_shot_id']]

print(df['possession'].unique())
print(df[df['pass_assisted_shot_id'].notnull()])
print(l)


#for i in l:
#  print(i, df['pass_assisted_shot_id'][i])

for i in l['possession']: 
    df.loc[df.possession == i, 'pass_shot_assist_x'] = df['pass_location_x']
    df.loc[df.possession == i, 'pass_shot_assist_y'] = df['pass_location_y']
    df.loc[df.possession == i, 'pass_shot_assist_end_x'] = df['pass_end_location_x']
    df.loc[df.possession == i, 'pass_shot_assist_end_y'] = df['pass_end_location_y']
    #df.loc[df.possession == i, 'assist_id'] = df['pass_assisted_shot_id'].mask("NaN", df['pass_assisted_shot_id'][i])

l = l.reset_index()
print(l)
for i in range(len(l)):
    print(l['possession'][i])
    df['pass_assisted_shot_id'][df['possession'] == l['possession'][i]]=l['pass_assisted_shot_id'][i]
    #df.iloc[df['possession'] == i]['pass_assisted_shot_id']=l[i]['possession']]=1
    #print(l.iloc[i][:])

#h = 0
#for i in range(len(df['pass_assisted_shot_id'])):
#    #print(df['possession'][i])
#    if df['possession'][i] in l['possession']:
#        df['pass_assisted_shot_id'][i] = l['pass_assisted_shot_id'][h]
#        h += 1
#print(df['pass_assisted_shot_id'])
#x = df['pass_assisted_shot_id'].groupby(df['possession'])

#for i,j in x:
#  print(i,j)

#print(x.possession)
#df['pass_assisted_shot_id'] = df['pass_assisted_shot_id'].groupby(df['possession']).transform(', '.join)
#df['jointed_size'] = df['size'].dropna().groupby(df['id']).transform(', '.join)
#df['pass_assisted_shot_id'] = df.groupby('possession').transform(lambda x: x)

#df = df.groupby('possession')['pass_assisted_shot_id']

#df["pass_assisted_shot_id"]=df["pass_assisted_shot_id"].ffill()
#df['pass_assisted_shot'] = df.groupby('possession'['pass_assisted_shot'].bfill())
#df['possession'] = df['possession'].fillna(df.groupby('possession')['pass_assisted_shot_id'])
#df['possession'].fillna(df.groupby(['period_id', 'gic_subindustry_id'], as_index=False).mean())    
#df['possession'].fillna(df.groupby(['possession'], as_index=False).mean())  
    #df['Final Rate']=df['Hourly Rate'].mask(pd.isnull, df['Daily Rate'])
#df
#1
#2
#df['Final Rate']=df['Hourly Rate'].mask(pd.isnull, df['Daily Rate'])
#df
    
    #df['pass_outcome_name'] = df['pass_outcome_name'].replace(np.nan, "Complete")
#df[df['shot_assist']==True]

#df['DataFrame Column'] = df['DataFrame Column'].replace(np.nan, 0)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(df[['pass_assisted_shot_id','possession']])
#print(df['id'].unique(), df['pass_assisted_shot_id'].unique())

#print(df[df['possession']==43])

df = df[['pass_location_x','pass_location_y', 'play_pattern_name', 'shot_statsbomb_xg',
         'shot_location_x','shot_location_y', 'shot_type_name', 'pass_type_name', 'shot_end_location_x',
         'shot_end_location_y', 'shot_end_location_z', 'pass_end_location_x', 'pass_end_location_y', 
         'possession', 'pass_shot_assist_x', 'pass_shot_assist_y', 'pass_shot_assist_end_x', 'pass_shot_assist_end_y','pass_assisted_shot_id', 'id']]

#print(df[df['possession'].notnull()])

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#print(df[df['pass_shot_assist_y'].notnull()])

#print(df.columns)

#for i in df.columns:
#  try:
#    print([df.keys() for df in df[df[i].notnull()][i]])
#  except:
#    pass

#print(df[['type','related_events']])
#df['play'] = 0
#j = 0
#for i, outcome in enumerate(df['outcome']):
#    if outcome == None:
#        df['play'][i] = j
#    else:
#        j += 1
#        df['play'][i] = j
#l = df[df['shot_assist']==True]['play'].to_list()
#for i in l: 
#    df.loc[df.play == i, 'shot_assist'] = True
#df = df[df['shot_assist']==True]
#print(vars(events))
#df = events.get_dataframe(event_type='shot').to_dict('index')
#df1 = events.get_dataframe(event_type='shot')
#df2 = events.get_dataframe(event_type='pass')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


[nan  9. 75. 77. 76. 74.]
['Complete' 'Incomplete' 'Out' 'Unknown' 'Pass Offside' 'Injury Clearance']
['id' 'index' 'period' 'timestamp' 'minute' 'second' 'possession'
 'duration' 'type_id' 'type_name' 'possession_team_id'
 'possession_team_name' 'play_pattern_id' 'play_pattern_name' 'team_id'
 'team_name' 'tactics_formation' 'tactics_lineup' 'related_events'
 'location' 'player_id' 'player_name' 'position_id' 'position_name'
 'pass_recipient_id' 'pass_recipient_name' 'pass_length' 'pass_angle'
 'pass_height_id' 'pass_height_name' 'pass_end_location' 'pass_type_id'
 'pass_type_name' 'pass_body_part_id' 'pass_body_part_name'
 'carry_end_location' 'under_pressure' 'pass_outcome_id'
 'pass_outcome_name' 'ball_receipt_outcome_id' 'ball_receipt_outcome_name'
 'duel_type_id' 'duel_type_name' 'pass_aerial_won' 'duel_outcome_id'
 'duel_outcome_name' 'counterpress' 'interception_outcome_id'
 'interception_outcome_name' 'pass_switch' 'pass_cross'
 'dribble_outcome_id' 'dribble_outcome_name' 'fou

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
#! pip install matplotlib
#! pip install mpld3
#! pip install --upgrade bokeh
from bokeh.plotting import figure, output_file, show, gridplot, ColumnDataSource
from bokeh.io import output_notebook, curdoc
from bokeh.models import LabelSet, Label, HoverTool, Text, CategoricalColorMapper, LinearInterpolator, Arrow, NormalHead, OpenHead, VeeHead
import matplotlib.pyplot as plt
from matplotlib.patches import Arc, Rectangle, ConnectionPatch
#import mpld3
from bokeh.transform import factor_cmap
import colorsys
import seaborn as sns
import pandas as pd
from bokeh.palettes import Viridis
from bokeh.palettes import RdBu3
#import bokeh.palettes

plays = []
legend = []
colors = {}
j = 78
#l = 5

def draw_pitch(l=5):

    p1 = figure(plot_width=int(1220/l), plot_height=int(820/l), tools='hover, lasso_select, box_select,reset')

    # add a circle renderer with a size, color, and alpha
    p1.rect(60, 40, width=120,height=80, color="white", line_color='black')
    p1.rect(9,40, width = 18, height = 44, color="white", line_color='black')
    #LeftPenalty = Rectangle([0,18], width = 18, height = 44, fill = False)
    p1.rect(111,40, width = 18, height = 44, color="white", line_color='black')
        #midline = ConnectionPatch([60,0], [60,80], "data", "data")

    p1.circle(60, y=40,size=20/l*8.1, line_color="black", fill_color="white")
    p1.circle(60, y=40,size=20/l*0.71, line_color="black", fill_color="black")
    p1.circle(12, y=40,size=20/l*0.71, line_color="black", fill_color="black")
    p1.circle(108, y=40,size=20/l*0.71, line_color="black", fill_color="black")

    p1.line([60,60], [80,0], color="black")
        #Left, Right 6-yard Box
    p1.rect(3,40, width = 6, height = 20, color="white", line_color='black')
    p1.rect(117,40, width = 6, height = 20, color="white", line_color='black')

    p1.arc(radius=10, start_angle=60, end_angle=(300),end_angle_units='deg',start_angle_units='deg', x=13,y=40, direction='clock',line_color='black')
    p1.arc(radius=10, start_angle=(120), end_angle=(240),end_angle_units='deg',start_angle_units='deg', x=107,y=40, direction='anticlock', line_color='black')

    return p1

def draw_goal():
    p = figure(plot_width=int(1220/l), plot_height=int(820/l), x_range=(34,46), y_range=(-1,3.67), tools='hover, lasso_select, box_select,reset')
 
    p.rect(40,1.335, width = 12, height = 4.67, fill_color = 'gray')
    p.rect(40, 1.335, width=8, height=2.67, fill_color='black', line_color='white')

    return p

def show_1(n,df,f,g,s, source):

    s_i = shots_in_pos(f,df,source)

    s_f = shots_fin_pos(g,df,source)

    s_p = passes(s,df, source)
    
    output_notebook()

    if n == 'field':

        return s_i
        
    elif n == 'goal':

        return s_f

    elif n =='passes':

        return s_p
    
    else:

        return gridplot([[s_p,None],[s_i,s_f]])

def shots_in_pos(p,df,source):
    #df1 = df[df['shot_type_name'].notnull()]
    #source = ColumnDataSource(df1)
    n = df['play_pattern_name'].unique()
    print(n)
    pal = Viridis[len(n)]
    
    color_mapper = CategoricalColorMapper(factors=sorted(df['play_pattern_name'].unique()), palette=pal)

    size_mapper=LinearInterpolator(
    x=[0,1],
    y=[10*3/l,50*3/l]
)
    #pd.set_option('display.max_columns', None)
    #print(df1)
    p.scatter(x='shot_location_x', y='shot_location_y',
          fill_color={'field': 'play_pattern_name', 'transform': color_mapper}, fill_alpha=0.6,
          size={'field':'shot_statsbomb_xg','transform': size_mapper},
          line_color=None, source=source)

    hover = p.select(dict(type=HoverTool))
 
    hover.tooltips = [("Player", "@player"),('Play',"@play_pattern"), ('Xg', '@statsbomb_xg{%0.2f}')]
    

    return p
def add_legend(p,df,x):
    global j

    if df['play_pattern'] not in legend:

        legend.append(df['play_pattern'])

        t = Label(x=103, y=j,text='fhnnsd', x_offset=0, y_offset=0, text_color=colors[df['play_pattern']], text_font_size='2pt')

        j -= 2
        p.add_layout(t)
    return p

def shots_fin_pos(p,df,source):
    #df1=df[df['shot_type_name'].notnull()]
    #source = ColumnDataSource(df1)
    n = df['play_pattern_name'].unique()
    print(n)
    pal = Viridis[len(n)]

    index_cmap = factor_cmap('play_pattern_name', palette=pal, 
                         factors=sorted(df['play_pattern_name'].unique()))
            
    p.scatter(x='shot_end_location_y', y='shot_end_location_z', radius=0.4,
        fill_color=index_cmap, fill_alpha=0.6,
        line_color=None, source=source)
        
    hover = p.select(dict(type=HoverTool))

    hover.tooltips = [("Outcome", "@outcome"),('Position',"@position")]

    return p

def passes(s, df, source):
    #df1 = df[df['pass_type_name'].notnull()]
    #source = ColumnDataSource(df1)
    #for i in df2['play'].unique():
    #    s = draw_pitch()
    #    df3 = df2[df2['play'] == i]
    #source = ColumnDataSource(df)
        
    s.scatter(x='pass_shot_assist_x', y='pass_shot_assist_y', source=source)

    s.add_layout(Arrow(end=VeeHead(size=5), source=source, x_start='pass_shot_assist_x', y_start='pass_shot_assist_y', x_end='pass_shot_assist_end_x', y_end='pass_shot_assist_end_y'))
        
    hover = s.select(dict(type=HoverTool))

    hover.tooltips = [("Play", "@play")]

    return s

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
from bokeh.models import LabelSet, Label, HoverTool, Text, CategoricalColorMapper, LinearInterpolator, Arrow, NormalHead, OpenHead, VeeHead, CategoricalMarkerMapper
from bokeh.plotting import figure, output_file, show, gridplot, ColumnDataSource

frames = df1[df1['shot_freeze_frame'].notnull()]['shot_freeze_frame']

df_frames = df1[df1['shot_freeze_frame'].notnull()]['location']
s_x = []
s_y = []
for i, j in df_frames:
    s_x.append(i)
    s_y.append(j)

f = {}
for i, frame in enumerate(frames):
    f[i] = json_normalize(frame, sep='_')
x = []
y = []
t = []
#print(f)
#color_mapper = CategoricalColorMapper(factors=sorted(frames['teammate'].unique()), palette=pal)

for i,j in f.items(): 
    for h,k in j.items():
        #print(i,h,k[0], k[1], k)
        #n = j['teammate'].unique()
        #pal = Viridis[len(n)]
        #print(h)
        color_mapper = CategoricalColorMapper(factors=['F','T', 'S'], palette=['blue','red','green'])
        if h == 'location':
            for _,g in k.items():
                x.append(g[0])
                y.append(g[1])
        elif h == 'teammate':
                
            for u,v in k.items():        
                #print(v)
                if v == False:
                    t.append('F')
                else:
                    t.append('T')
    t.append('S')
    x.append(s_x[i])
    y.append(s_y[i])
    d = {'x' : x, 'y': y, 'teammate':t}
    source = ColumnDataSource(data=d)
    #print(x,y,t)
    p = draw_pitch()
    #source = ColumnDataSource(data=(x=x,y=y))
    p.scatter(x='x', y='y',
              fill_color={'field': 'teammate', 'transform': color_mapper}, source=source,
              line_color=None, fill_alpha=0.6, size = 6
              )
    output_notebook()
    curdoc().add_root(p)
    show(p)
    x = []
    y = []
    t = []
    #print(j['location'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


In [7]:
print(f[0])
df_frames = df1[df1['shot_freeze_frame'].notnull()]['location']
s_x = []
s_y = []
for i, j in df_frames:
    s_x.append(i)
    s_y.append(j)
    
    #print(i,j)
print(s_x,s_y)

         location  teammate  player_id           player_name  position_id  \
0   [120.0, 42.0]     False       3099           Hugo Lloris            1   
1    [99.0, 33.0]      True       5470          Ivan Rakitić           15   
2   [112.0, 33.0]      True       5465            Ante Rebić           17   
3   [111.0, 38.0]      True       5474          Ivan Perišić           21   
4   [113.0, 41.0]      True       3471          Dejan Lovren            3   
5   [113.0, 43.0]      True       5472       Mario Mandžukić           23   
6   [113.0, 33.0]     False       5484    Lucas Hernández Pi            6   
7   [113.0, 39.0]     False       5476       Benjamin Pavard            2   
8   [113.0, 52.0]     False       5487     Antoine Griezmann           22   
9   [110.0, 47.0]     False       5492    Samuel Yves Umtiti            5   
10  [113.0, 46.0]     False       3604        Olivier Giroud           24   
11  [102.0, 37.0]     False       3009  Kylian Mbappé Lottin           12   

In [8]:
f[0].columns

Index(['location', 'teammate', 'player_id', 'player_name', 'position_id',
       'position_name'],
      dtype='object')

In [33]:
frames = df1[df1['shot_freeze_frame'].notnull()]['shot_freeze_frame']

f = {}
#for i, frame in enumerate(frames):
    #print(frame)
#    f[i] = json_normalize(frame, sep='_')



plays = df1[df1['shot_freeze_frame'].notnull()]
poss = plays['possession'].values
#print(type(plays))
#print(len(plays))
#print(poss)
for i in range(len(plays)):
    f = json_normalize(plays.iloc[i]['shot_freeze_frame'], sep='_')
    #print(plays['possession'].values)
    p = df1[df1['possession']==poss[i]]
    df_f = f.append(p)
    
    
    df_f[['location_x', 'location_y']] = df_f['location'].apply(pd.Series)
    df_f.loc[(df_f['team_name'] != df_f['possession_team_name']) & (df_f['team_name'].notnull()),  'location_x'] = 120-df_f['location_x']
    df_f.loc[(df_f['team_name'] != df_f['possession_team_name']) & (df_f['team_name'].notnull()),  'location_y'] = 80-df_f['location_y']
    df_f.loc[df_f['team_name'] == df_f['possession_team_name'], 'T'] = 'Attacker'
    df_f.loc[df_f['team_name'] != df_f['possession_team_name'], 'T'] = 'Deffender'
    df_f.loc[df_f['teammate'].notnull(),'P'] = 'Shot'
    df_f.loc[df_f['P'] != 'Shot','P'] = 'Play'
    df_f.loc[(df_f['pass_assisted_shot_id'].notnull()) | (df_f['shot_body_part_name'].notnull()), 'P'] = 'Shot'
    df_f.loc[df_f['team_name'] != df_f['possession_team_name'], 'T'] = 'Deffender'
    df_f.loc[df_f['teammate'] == True, 'T'] = 'Attacker'
    #df_f.loc[(df_f['teammate'] == False) and (df_f.loc[df_f['position_name'] != 'Goalkeeper'), 'T'] = 'Deffender'
    df_f.loc[(df_f['teammate'] == False) & (df_f['position_name'] != 'Goalkeeper'), 'T'] = 'Deffender'
    df_f.loc[(df_f['teammate'] == False) & (df_f['position_name'] == 'Goalkeeper'), 'T'] = 'Goalkeeper'
    df_f.loc[(df_f['shot_body_part_name'].notnull()), 'T'] = 'Shot'
    df_f.loc[(df_f['pass_assisted_shot_id'].notnull()), 'T'] = 'Assist'
    #df.loc[(df['First_name'] != 'Bill') & (df['First_name'] != 'Emma'), 'name_match'] = 'Mismatch'  
    #print(df_f)
    color_mapper = CategoricalColorMapper(factors=['Attacker','Deffender', 'Goalkeeper', 'Shot', 'Assist'], palette=['blue','red', 'yellow', 'green', 'orange'])
    marker_mapper = CategoricalMarkerMapper(factors=['Play','Shot'], markers=['triangle','circle'])
    #CategoricalMarkerMapper
    #f[0]
    #df2 = df1.iloc[723]
    #pd.concat([f[0],df2], axis=0)
    l = 1
    pl = draw_pitch(l)

    source = ColumnDataSource(df_f)

    pl.scatter(x='location_x', y='location_y', 
              fill_color={'field': 'T', 'transform': color_mapper}, source=source,
              line_color=None, fill_alpha=0.6, size = 30/l,
              marker={'field': 'P', 'transform': marker_mapper}
              )
    show(pl)
    
    
    
    
    
    
    #print(d)
    #print(f)  
    #frame = json_normalize(frame, sep='_')
#print(plays)
#print(df1[df1['shot_body_part_name'].notnull()])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


In [10]:
f = {}
for i, frame in enumerate(frames):
    f[i] = json_normalize(frame, sep='_')
print(df1[700:725])
df2 = df1.iloc[723]
df3 = df1.iloc[721]
df4 = df1[df1['possession']==42]
#df4 = df1.iloc[700:7]
df_f=f[0].append(df2).append(df3).append(df4)

df_f[['location_x', 'location_y']] = df_f['location'].apply(pd.Series)
df_f.loc[(df_f['team_name'] != df_f['possession_team_name']) & (df_f['team_name'].notnull()),  'location_x'] = 120-df_f['location_x']
df_f.loc[(df_f['team_name'] != df_f['possession_team_name']) & (df_f['team_name'].notnull()),  'location_y'] = 80-df_f['location_y']
df_f.loc[df_f['team_name'] == df_f['possession_team_name'], 'T'] = 'Attacker'
df_f.loc[df_f['team_name'] != df_f['possession_team_name'], 'T'] = 'Deffender'
df_f.loc[df_f['teammate'].notnull(),'P'] = 'Shot'
df_f.loc[df_f['P'] != 'Shot','P'] = 'Play'
df_f.loc[(df_f['pass_assisted_shot_id'].notnull()) | (df_f['shot_body_part_name'].notnull()), 'P'] = 'Shot'
df_f.loc[df_f['team_name'] != df_f['possession_team_name'], 'T'] = 'Deffender'
df_f.loc[df_f['teammate'] == True, 'T'] = 'Attacker'
#df_f.loc[(df_f['teammate'] == False) and (df_f.loc[df_f['position_name'] != 'Goalkeeper'), 'T'] = 'Deffender'
df_f.loc[(df_f['teammate'] == False) & (df_f['position_name'] != 'Goalkeeper'), 'T'] = 'Deffender'
df_f.loc[(df_f['teammate'] == False) & (df_f['position_name'] == 'Goalkeeper'), 'T'] = 'Goalkeeper'
df_f.loc[(df_f['shot_body_part_name'].notnull()), 'T'] = 'Shot'
df_f.loc[(df_f['pass_assisted_shot_id'].notnull()), 'T'] = 'Assist'
#df.loc[(df['First_name'] != 'Bill') & (df['First_name'] != 'Emma'), 'name_match'] = 'Mismatch'  
print(df_f)
color_mapper = CategoricalColorMapper(factors=['Attacker','Deffender', 'Goalkeeper', 'Shot', 'Assist'], palette=['blue','red', 'yellow', 'green', 'orange'])
marker_mapper = CategoricalMarkerMapper(factors=['Play','Shot'], markers=['triangle','circle'])
#CategoricalMarkerMapper
#f[0]
#df2 = df1.iloc[723]
#pd.concat([f[0],df2], axis=0)
l = 1
pl = draw_pitch(l)

source = ColumnDataSource(df_f)

pl.scatter(x='location_x', y='location_y', 
           fill_color={'field': 'T', 'transform': color_mapper}, source=source,
           line_color=None, fill_alpha=0.6, size = 30/l,
           marker={'field': 'P', 'transform': marker_mapper}
           )
show(pl)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


                                       id  index  period     timestamp  \
700  df2921c1-acfa-4edc-8ab9-15de3d9e57a2    701       1  00:19:39.307   
701  ccf732d9-58af-4379-ba3d-c47dcc50ebdd    702       1  00:19:40.760   
702  5f5f0329-c15f-4c5b-bb18-380fcdd7baaa    703       1  00:19:40.760   
703  4b550c89-3546-40b3-9041-4acfdf04544a    704       1  00:19:41.907   
704  15d564fa-1f82-4c5e-9850-9d4da064bd5f    705       1  00:19:42.867   
705  af3e85a8-ed0c-4b51-8728-472ff9f6eee9    706       1  00:19:42.867   
706  b0bb9eca-2e58-4860-81d3-ab3ed924eab5    707       1  00:19:44.067   
707  778b34f9-9531-4648-8f64-8a7716e88030    708       1  00:19:44.067   
708  d5e9221e-1c1b-4e0c-bc59-930007615cb4    709       1  00:19:44.680   
709  e5cd25ae-3dfc-4489-9490-d2f94658cee7    710       1  00:19:45.267   
710  cae92f86-08d3-4cdc-9502-7be98a185057    711       1  00:19:45.867   
711  6ce091a7-322a-4135-b2f9-eee45fd2e0dc    712       1  00:19:45.867   
712  7e59eb43-42a7-4cf4-99c9-55592d496

In [11]:
df_frames = df1[df1['shot_freeze_frame'].notnull()]
print(df_frames)

                                        id  index  period     timestamp  \
723   cfdb23f5-0247-4f10-9641-8ac06824722f    724       1  00:20:51.227   
792   2776f49c-73a8-4185-bcce-9cbef244cf3c    793       1  00:23:34.907   
910   dd524196-386d-4759-b40b-e920cc77b059    911       1  00:27:53.880   
1076  b99d698f-5891-402b-b5cf-33ca81e1554f   1077       1  00:39:25.267   
1195  882663ad-d2a6-4cc3-81af-8efcbf7a8306   1196       1  00:42:47.027   
1200  f1836373-a551-4937-a4b4-5dbaa5c408ec   1201       1  00:42:50.027   
1230  a9cf99b0-5555-4ef1-81f9-e33360cee69d   1231       1  00:45:17.307   
1404  474afda2-d8c7-4254-93bf-895d9c440ccf   1405       2  00:01:10.027   
1458  f6f46328-e8a4-41b6-b30f-96996ba46707   1459       2  00:02:02.947   
1463  d658768b-35c7-4aa5-85c4-f1fa71d37e3c   1464       2  00:02:32.627   
1544  3bba8f94-7feb-43f6-b624-2d305275a74a   1545       2  00:04:34.320   
1647  cc69d1e5-36bf-4522-ae6d-fa1459dfb877   1648       2  00:06:35.947   
1858  73b228b2-3719-4d37-

In [12]:
f = draw_pitch()
g = draw_goal()
s = draw_pitch()
source = ColumnDataSource(df)
a = show_1('passs',df, f, g ,s ,source)
curdoc().add_root(a)
show(a)

['Regular Play' 'From Kick Off' 'From Throw In' 'From Goal Kick'
 'From Free Kick' 'From Corner' 'From Counter' 'From Keeper' 'Other']
['Regular Play' 'From Kick Off' 'From Throw In' 'From Goal Kick'
 'From Free Kick' 'From Corner' 'From Counter' 'From Keeper' 'Other']


In [13]:
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import CustomJS
from bokeh.plotting import figure

p1 = figure(plot_width=300, plot_height=300, tools="tap")
r1 = p1.circle(x=[1, 2], y=1, color=["red", "blue"], size=20)

p2 = figure(plot_width=300, plot_height=300, tools="")
r2 = p2.circle(x=[1, 1, 2], y=[1, 2, 1.5], color=["red", "red", "blue"], size=20)

callback = CustomJS(args=dict(s2=r2.data_source), code="""
const s2_inds = []
if (cb_obj.indices.indexOf(0) >= 0) {
  s2_inds.push(0)
  s2_inds.push(1)
}
if (cb_obj.indices.indexOf(1) >= 0) {
  s2_inds.push(2)
}
s2.selected.indices = s2_inds
""")

show(row(p1, p2))

In [14]:
#Based on Tony's answer (using BoxSelectTool for single selection), I came up with code for multiple selection. (Note that I made some typo's in my original callback1 function)

from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter, CustomJS, BoxSelectTool
import pandas as pd
# output_notebook()
data = {'person': [1, 1, 1, 2, 2, 3, 3, 3, 3],
        'activities':['a', 'b', 'c', 'a', 'b', 'a', 'b', 'c', 'd'],
        'hours':[3, 4, 6, 2, 7, 5, 3, 2, 12],
        'foodeaten':[12, 14, 34, 45, 67, 5, -1, 3, 5]}
df = pd.DataFrame(data = data)
print(df)
source = ColumnDataSource(data = df)
views = [(df.activities == l) for l in ['a', 'b', 'c', 'd']]
filtered_views = [CDSView(source = source, filters = [BooleanFilter(view.values.tolist())]) for view in views]
plot_options = dict(plot_width = 250,
                    plot_height = 250,
                    tools = "box_select,tap,pan,wheel_zoom,reset,save",
                    tooltips = [("Person", "@person"), ("hours", "@hours")])
plots = [figure(title = 'activity {l}'.format(l = l), **plot_options) for l in ['a', 'b', 'c', 'd', 'a', 'b', 'c', 'd']]
for plot, view, name in zip(plots, 2 * filtered_views, 4 * ['hours'] + 4 * ['foodeaten']):
    plot.circle('person', y = name, size = 15, view = view, source = source)


callback1 = CustomJS(args = dict(source = source, ), code = """
var selected_indices = source.selected.indices;
var data = source.data;
debugger;
console.log(selected_indices)
var all_selected = [];
var persons = [];

for (i in selected_indices){
    index = selected_indices[i];
    console.log(data['person'][index]);
    persons.push(data['person'][index]);
    }




    }
    }
source.selected.indices=all_selected;
""")

# output_file('testSelect.html')
for plot in plots:
    plot.select(BoxSelectTool).callback = callback1
show(gridplot(children = [plot for plot in plots], ncols = 4))

   person activities  hours  foodeaten
0       1          a      3         12
1       1          b      4         14
2       1          c      6         34
3       2          a      2         45
4       2          b      7         67
5       3          a      5          5
6       3          b      3         -1
7       3          c      2          3
8       3          d     12          5


In [15]:
import os
from bokeh.models.selections import Selection
from bokeh.models import ColumnDataSource, Patches, CustomJS
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.io import output_file, curdoc
import pandas as pd

#def app(doc):
x = [[1,2,4], [3,5,6], [7,9,7], [5,7,6]]
y = [[4,2,1], [6,5,8], [3,9,6], [2,2,1]]
group = ['A', 'A', 'B', 'B']
id = [0,1,2,3]

df = pd.DataFrame(data=dict(x=x, y=y, group=group, id=id))
source = ColumnDataSource(df)

p = figure(tools="tap")

renderer = p.patches('x', 'y', source=source)

def my_tap_handler(attr,old,new):
    indices = source.selected.indices
    if len(indices) == 1:
        group = source.data["group"][indices[0]]
        new_indices = [i for i, g in enumerate(source.data["group"]) if g == group]
    if new_indices != indices:
        source.selected = Selection(indices=new_indices)

selected_patches = Patches(fill_color="#a6cee3")
renderer.selection_glyph = selected_patches
source.on_change("selected", my_tap_handler)
output_notebook()
curdoc().add_root(row(p, width=800))

show(p)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [16]:
from bokeh.plotting import figure, output_file
from bokeh.models import CustomJS, HoverTool, CheckboxGroup
from bokeh.layouts import gridplot
from bokeh.resources import CDN
from bokeh.embed import file_html
fig = figure(plot_width=400,plot_height=400,toolbar_location=None
)
plots = []
plots.append( fig.line(x=range(10),y=range(10),name='first') )
plots.append( fig.line(x=range(10),y=range(10)[::-1],name='second') )
plots.append( fig.line(x=range(10),y=[i**0.5 for i in range(10)],name='third'
) )
hover = HoverTool(mode='vline',line_policy='next'
,names=[])
fig.add_tools(hover)
labels = ['first','second','third'
]
N_plots = range(len(plots))
#checkbox = CheckboxGroup(labels=labels, active=[],width=200
#)
js_args = {
'p0': plots[0    ],
'p1': plots[1    ],
'p2': plots[2    ],
'fig'    : fig,
}
checkbox_code ="""

var indexOf = [].indexOf;
var new_renderers = []
if(indexOf.call(0) >= 0) new_renderers.push(p0);
if(indexOf.call(1) >= 0) new_renderers.push(p1);
if(indexOf.call(2) >= 0) new_renderers.push(p2);
fig.renderers = new_renderers;
"""

checkbox.callback = CustomJS(args=js_args, code=checkbox_code)
grid = gridplot([[fig,checkbox]],toolbar_location='left'
)
show(grid)
#outfile=open('hovtest.html','w')
#outfile.write(file_html(grid,CDN,'test'))
#outfile.close()


NameError: ignored

In [0]:
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter, CustomJS
import pandas as pd

data = {'person': [1, 1, 1, 2, 2, 3, 3, 3, 3], 'activities':['a', 'b', 'c', 'a', 'b', 'a', 'b', 'c', 'd'], 'hours':[3, 4, 6, 2, 7, 5, 3, 2, 12], 'foodeaten':[12, 14, 34, 45, 67, 5, -1, 3, 5]}
df = pd.DataFrame(data = data)
source = ColumnDataSource(data = df)
views = [(df.activities == l) for l in ['a', 'b', 'c', 'd']]
filtered_views = [CDSView(source = source, filters = [BooleanFilter(view.values.tolist())]) for view in views]
plot_options = dict(plot_width = 250, plot_height = 250, tools = "tap,pan,wheel_zoom,reset,save", tooltips = [("Person", "@person"), ("hours", "@hours")])
plots = [figure(title = 'activity {l}'.format(l = l), **plot_options) for l in ['a', 'b', 'c', 'd', 'a', 'b', 'c', 'd']]
[plot.circle('person', y = name, size = 15, view = view, source = source) for plot, view, name in zip(plots, 2 * filtered_views, 4 * ['hours'] + 4 * ['foodeaten'])]
callback = CustomJS(args = dict(source = source, plots = plots), code = """
const selected_index = source.selected.indices[0]
const person = source.data['person'][selected_index]
var all_selected = [];
for (index in source.data['index']){
    if (source.data['person'][index] == person)
    all_selected.push(index) 
}
source.selected.indices = all_selected; """)

[plot.js_on_event('tap', callback) for plot in plots]
show(gridplot(children = [plot for plot in plots], ncols = 2))

In [0]:
df1 = df5[['shot_location_x','shot_location_y']]
#print(df1.head())
p = draw_pitch()
source = ColumnDataSource(df1)
#p.scatter(x=df1['shot_location_x'], y=df1['shot_location_y'])
p.scatter(x='shot_location_x', y='shot_location_y', source=source)
output_notebook()
curdoc().add_root(p)
show(p)

In [0]:
p = draw_pitch()
df2 = df[['pass_location_x','pass_location_y', 'play_pattern_name']]
source = ColumnDataSource(df2)
#p.scatter(x=df1['pass_location_x'], y=df1['pass_location_y'])
p.scatter(x='pass_location_x', y='pass_location_y', source=source)
output_notebook()
curdoc().add_root(p)
show(p)

In [0]:
#df2 = events.get_dataframe(event_type='pass')
#df2['shot_assist'].unique()


In [0]:
df2 = events.get_dataframe(event_type='pass')

s = draw_pitch()

df2 = df2[:10]

source = ColumnDataSource(df2)
#print(source.columns)
s.scatter(x=df2['start_location_x'], y=df2['start_location_y'])



s.add_layout(Arrow(end=VeeHead(size=5), source=source, x_start='start_location_x', y_start='start_location_y', x_end='end_location_x', y_end='end_location_y'))


show(s)


In [0]:
#import enumerate
pd.set_option("max_rows", None)
df3 = events.get_dataframe(event_type='pass')
df3['play'] = 0
j = 0
for i, outcome in enumerate(df3['outcome']):
    if outcome == None:
        df3['play'][i] = j
    else:
        j += 1
        df3['play'][i] = j
#print(df3[['outcome', 'play']])

for i in df3['play'].unique():
    s = draw_pitch()
    df4 = df3[df3['play'] == i]
    source = ColumnDataSource(df4)
#print(source.columns)
    s.scatter(x=df4['start_location_x'], y=df4['start_location_y'])



    s.add_layout(Arrow(end=VeeHead(size=5), source=source, x_start='start_location_x', y_start='start_location_y', x_end='end_location_x', y_end='end_location_y'))
    show(s)

    

In [0]:
df5 = df3.groupby('play').tail(1)
for i in df5['assisted_shot_id']==True:
    #print(df5['assisted_shot_id']==True)
    #if df5['assisted_shot_id'][i] == True:
    s = draw_pitch()
    df4 = df3[df3['play'] == i]
    source = ColumnDataSource(df4)
    #print(source.columns)
    s.scatter(x=df4['start_location_x'], y=df4['start_location_y'])



    s.add_layout(Arrow(end=VeeHead(size=5), source=source, x_start='start_location_x', y_start='start_location_y', x_end='end_location_x', y_end='end_location_y'))
    show(s)

In [0]:
print(df3.columns)
#df5 = df3.groupby('play').tail(1)
#df3[df3['play'].iloc[-1] == 0]
#print(a)
#stock_pick['Regime'].iloc[-1] = 0

In [0]:
from bokeh.plotting import figure, output_file, show, gridplot
from bokeh.io import output_notebook
from math import radians

# output to static HTML file
#output_file("line.html")

p = figure(plot_width=int(1220/5), plot_height=int(820/5))


# add a circle renderer with a size, color, and alpha
p.rect(60, 40, width=120,height=80, color="white", line_color='black')
p.rect(9,40, width = 18, height = 44, color="white", line_color='black')
#LeftPenalty = Rectangle([0,18], width = 18, height = 44, fill = False)
p.rect(111,40, width = 18, height = 44, color="white", line_color='black')
    #midline = ConnectionPatch([60,0], [60,80], "data", "data")

p.circle(60, y=40,size=20/5*8.1, line_color="black", fill_color="white")
p.circle(60, y=40,size=20/5*0.71, line_color="black", fill_color="black")
p.circle(12, y=40,size=20/5*0.71, line_color="black", fill_color="black")
p.circle(108, y=40,size=20/5*0.71, line_color="black", fill_color="black")

p.line([60,60], [80,0], color="black")
    #Left, Right 6-yard Box
p.rect(3,40, width = 6, height = 20, color="white", line_color='black')
p.rect(117,40, width = 6, height = 20, color="white", line_color='black')

p.arc(radius=10, start_angle=60, end_angle=(300),end_angle_units='deg',start_angle_units='deg', x=13,y=40, direction='clock',line_color='black')
p.arc(radius=10, start_angle=(120), end_angle=(240),end_angle_units='deg',start_angle_units='deg', x=107,y=40, direction='anticlock', line_color='black')

#p.arc(13,40, 16.2, color="white", line_color='black')
#p.arc(117,40, width = 6, height = 20, color="white", line_color='black')
    #LeftSixYard = Rectangle([0,30], width = 6, height = 20, fill = False)
    #RightSixYard = Rectangle([114,30], width = 6, height = 20, fill = False)
for i in range(len(df)):
        col = colors[df['play_pattern'][i]]
        d = tuple([255*x for x in col[:3]])
        t = Label(x=df['start_location_x'][i], y=df['start_location_y'][i],text=str(round(df['statsbomb_xg'][i],2)), 
            x_offset=0, y_offset=0, text_color=d, text_font_size='6pt')
        p.add_layout(t)
        
        #t = LabelSet(x=str(df['start_location_x'][i]), y=str(df['start_location_y'][i]),text=str(round(df['statsbomb_xg'][i],2)), level = 'glyph')
        #p.add_layout(t)

    #Prepare Circles
#p.Circle((60,40),8.1,color="white", line_color='black')
#    centreCircle = plt.Circle((60,40),8.1,color="black", fill = False)
#    centreSpot = plt.Circle((60,40),0.71,color="black")
    #Penalty spots and Arcs around penalty boxes
#    leftPenSpot = plt.Circle((12,40),0.71,color="black")
#    rightPenSpot = plt.Circle((108,40),0.71,color="black")
#    leftArc = Arc((13,40),height=16.2,width=16.2,angle=0,theta1=310,theta2=50,color="black")
#    rightArc = Arc((107,40),height=16.2,width=16.2,angle=0,theta1=130,theta2=230,color="black")
    
output_notebook()
# show the results
show(p)

In [0]:
p = figure(plot_width=int(1220/5), plot_height=int(820/5))
 
p.rect(40,1.335, width = 12, height = 4.67, fill_color = 'gray')
p.rect(40, 1.335, width=8, height=2.67, fill_color='black', line_color='white')
x = []
y = []
z = []
#mytext = Label(x=36, y=-1, text='here your text')

#p.add_layout(mytext)
for i in range(len(df)):
  if df['end_location_z'][i] >= -1 and df['end_location_z'][i] <= 3.67 and df['end_location_y'][i] >= 34 and df['end_location_y'][i] <= 46:
    col = colors[df['play_pattern'][i]]
    d = tuple([255*x for x in col[:3]])
    x=(df['end_location_y'][i])
    y=(df['end_location_z'][i])
    z=(round(df['statsbomb_xg'][i],2))
    t = Label(x=df['end_location_y'][i], y=df['end_location_z'][i],text=str(round(df['statsbomb_xg'][i],2)), 
            x_offset=0, y_offset=0,text_color=d, text_font_size='6pt')
              #x_offset=5, y_offset=5, render_mode='canvas')
    p.add_layout(t)
    #output_notebook()
    #show(p)
    #plt.text(df['end_location_y'][i], df['end_location_z'][i],round(df['statsbomb_xg'][i],2) , color=col, fontsize=50)
#goal = draw_goal(ay) #overlay our different objects on the pitch
print(x,y,z)
#t = LabelSet(x=str(df['start_location_x'][i]), y=str(df['start_location_y'][i]),text=str(round(df['statsbomb_xg'][i],2)), level = 'glyph')
#f.add_layout(t)

output_notebook()
# show the results
show(p)

In [0]:
p1 = figure(plot_width=int(1220/5), plot_height=int(820/5))


# add a circle renderer with a size, color, and alpha
p1.rect(60, 40, width=120,height=80, color="white", line_color='black')
p1.rect(9,40, width = 18, height = 44, color="white", line_color='black')
#LeftPenalty = Rectangle([0,18], width = 18, height = 44, fill = False)
p1.rect(111,40, width = 18, height = 44, color="white", line_color='black')
    #midline = ConnectionPatch([60,0], [60,80], "data", "data")

p1.circle(60, y=40,size=20/5*8.1, line_color="black", fill_color="white")
p1.circle(60, y=40,size=20/5*0.71, line_color="black", fill_color="black")
p1.circle(12, y=40,size=20/5*0.71, line_color="black", fill_color="black")
p1.circle(108, y=40,size=20/5*0.71, line_color="black", fill_color="black")

p1.line([60,60], [80,0], color="black")
    #Left, Right 6-yard Box
p1.rect(3,40, width = 6, height = 20, color="white", line_color='black')
p1.rect(117,40, width = 6, height = 20, color="white", line_color='black')

p1.arc(radius=10, start_angle=60, end_angle=(300),end_angle_units='deg',start_angle_units='deg', x=13,y=40, direction='clock',line_color='black')
p1.arc(radius=10, start_angle=(120), end_angle=(240),end_angle_units='deg',start_angle_units='deg', x=107,y=40, direction='anticlock', line_color='black')

p2 = figure(plot_width=int(1220/5), plot_height=int(820/5))
 
p2.rect(40,1.335, width = 12, height = 4.67, fill_color = 'gray')
p2.rect(40, 1.335, width=8, height=2.67, fill_color='black', line_color='white')

p = gridplot([[p1],[p2]])

output_notebook()
# show the results
show(p)

In [0]:
fig=plt.figure() #set up the figures
fig.set_size_inches(49, 70)
ax=fig.add_subplot(2,1,1)
for i in range(len(df)):
  col = colors[df['play_pattern'][i]]
  plt.text(df['start_location_x'][i], df['start_location_y'][i],round(df['statsbomb_xg'][i],2) , color=col, fontsize=50)
j = 2
for i in range(len(plays)):
  plt.text(103, j,plays[i] , color=pal[i], fontsize=50)
  j += 2
field = draw_pitch(ax) #overlay our different objects on the pitch
plt.ylim(-1, 81)
plt.xlim(-1, 121)
plt.gca().invert_yaxis()
plt.axis('off')

ay=fig.add_subplot(2,1,2)
ay.set_facecolor('xkcd:gray')
ay.set_yticklabels([])
ay.set_xticklabels([])
for i in range(len(df)):
  if df['end_location_z'][i] >= -1 and df['end_location_z'][i] <= 3.67 and df['end_location_y'][i] >= 34 and df['end_location_y'][i] <= 46:
    col = colors[df['play_pattern'][i]]
    plt.text(df['end_location_y'][i], df['end_location_z'][i],round(df['statsbomb_xg'][i],2) , color=col, fontsize=50)
goal = draw_goal(ay) #overlay our different objects on the pitch


mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(goal, labels=col))

plt.ylim(-1, 3.67)
plt.xlim(34, 46)
plt.gca().axes.get_yaxis().set_visible(False)
plt.gca().axes.get_xaxis().set_visible(False)
#plt.gca().invert_yaxis()
#plt.axis('off')
#plt.axes.get_yaxis().set_visible(False)
mpld3.display()


#fig, ax = plt.subplots()
#fig.set_size_inches(7, 5)
#fig.set_size_inches(49, 35)
#for i in range(len(df)):
#  col = colors[df['play_pattern'][i]]
  #print(play)
#  plt.text(df['start_location_x'][i], df['start_location_y'][i],round(df['statsbomb_xg'][i],2) , color=col, fontsize=50)
#j = 2
#for i in range(len(plays)):
#  plt.text(103, j,plays[i] , color=pal[i], fontsize=50)
#  j += 2
#field = draw_pitch(ax) #overlay our different objects on the pitch

#plt.ylim(-2, 82)
#plt.xlim(-2, 122)
#plt.gca().invert_yaxis()
#plt.axis('off')

#plt.show()